In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps阶段2：实验

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/mlops_experimentation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/mlops_experimentation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

*注意：该笔记本不支持在Colab中执行*

## 概述

本教程演示了如何在 Google Cloud 上使用 Vertex AI 进行端到端的生产环境 MLOps。本教程涵盖了阶段 2：实验。

### 目标

在本教程中，您将创建一个MLOps阶段2：实验过程。

本教程使用以下Vertex AI：

- `Vertex AI Datasets`
- `Vertex AI Models`
- `Vertex AI AutoML`
- `Vertex AI Training`
- `Vertex AI TensorBoard`
- `Vertex AI Vizier`
- `Vertex AI Batch Prediction`

执行的步骤包括：

- 查看在阶段1创建的 `Dataset` 资源。
- 在后台训练AutoML表格二元分类器模型。
- 构建实验性模型架构。
- 为 `Dataset` 资源构建自定义训练包。
- 在本地测试自定义训练包。
- 使用Vertex AI Training在云端测试自定义训练包。
- 使用Vertex AI Vizier对模型训练进行超参数调整。
- 使用Vertex AI Training训练自定义模型。
- 为自定义模型添加用于在线/批处理预测的服务函数。
- 使用服务函数测试自定义模型。
- 使用Vertex AI Batch Prediction评估自定义模型。
- 等待AutoML训练作业完成。
- 使用相同的评估片段使用Vertex AI Batch Prediction评估AutoML模型。
- 将AutoML模型的评估结果设置为基准。
- 如果自定义模型的评估低于基准，请继续对自定义模型进行实验。
- 如果自定义模型的评估高于基准，请将模型保存为第一个最佳模型。

### 推荐

在谷歌云上进行端到端（E2E）MLOps实验时，建议采用以下关于结构化（表格）数据的最佳实践：

 - 使用AutoML确定基线评估。
 - 设计并构建模型架构。
     - 将未经训练的模型架构上传为Vertex AI模型资源。


 - 构建一个可以在本地运行和作为Vertex AI训练作业运行的训练包。
     - 将训练包分解为：数据、模型、训练和任务Python模块。
     - 从Vertex AI数据集资源的用户元数据获取转换后的训练数据位置。
     - 从Vertex AI模型资源获取模型工件的位置。
     - 在训练包中包括初始化Vertex AI实验和相应运行。
     - 记录实验的超参数和训练参数。
     - 添加用于提前停止、TensorBoard和超参数调整的回调函数，其中超参数调整是一个命令行选项。


 - 用少量的时期在本地测试训练包。
 - 使用Vertex AI训练测试训练包。
 - 使用Vertex AI超参数调整进行超参数调整。
 - 使用Vertex AI训练对定制模型进行全面训练。
     - 记录实验/运行的超参数值。


 - 评估定制模型。
     - 单一评估切片，与AutoML相同的指标
         - 将评估添加到训练包中，并在用于训练的云存储存储桶中返回结果文件
     - 定制评估切片，定制指标
         - 将定制评估切片作为AutoML和定制模型的Vertex AI批量预测
         - 对批处理作业的结果执行定制指标


 - 将定制模型指标与AutoML基线进行比较
     - 如果低于基线，则继续实验
     - 如果高于基线，则将模型上传为新的基线，并将评估结果与模型保存。

### 数据集

本教程使用的数据集是[芝加哥出租车](https://www.kaggle.com/chicago/chicago-taxi-trips-bq)数据集。本教程中使用的数据集版本存储在一个公共BigQuery表中。训练好的模型可以预测一个人是否会给出租车费小费。

### 成本

此教程使用 Google Cloud 的计费组件：

* BigQuery
* Vision API
* Vertex AI
* Cloud Storage

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing)，[Vision API 价格](https://cloud.google.com/vision/pricing)，[BigQuery 价格](https://cloud.google.com/bigquery/pricing)，[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装执行MLOps笔记本所需的软件包*仅需一次*。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

ONCE_ONLY = False
if ONCE_ONLY:
    ! pip3 install -U {USER_FLAG} -q tensorflow==2.5 \
                                     tensorflow-data-validation==1.2 \
                                     tensorflow-transform==1.2 \
                                     tensorflow-io==0.18 
    
    ! pip3 install --upgrade {USER_FLAG} -q google-cloud-aiplatform[tensorboard] \
                                            google-cloud-pipeline-components \
                                            google-cloud-bigquery \
                                            google-cloud-logging \
                                            apache-beam[gcp] \
                                            pyarrow \
                                            cloudml-hypertune

重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必须的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用额度用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行前缀为 `!` 的行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

将您的项目ID设置为

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不能使用多区域存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}
if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试账户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经通过验证。跳过这一步。

**如果您正在使用 Colab**，运行下面的单元格并按照提示进行身份验证，使用 oAuth。

**否则**，请遵循以下步骤：

1. 在 Cloud Console 中，转到 [**创建服务帐号密钥** 页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击 **创建服务帐号**。

3. 在 **服务帐号名称** 字段中输入一个名称，然后点击 **创建**。

4. 在 **授予该服务帐号对项目的访问权限** 部分，点击 **Role** 下拉列表。在过滤框中输入 "Vertex AI"，并选择 **Vertex AI 管理员**。在过滤框中输入 "Storage Object Admin"，并选择 **Storage Object Admin**。

5. 点击 *创建*。包含您密钥的 JSON 文件将下载到您的本地环境中。

6. 在下面的单元格中输入您的服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS` 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，下面的步骤都是必需的。**

在初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源在各个会话中保留的地方。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对您的云存储存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

**服务账户**

**如果你不知道你的服务账户**，尝试使用`gcloud`命令执行下面的第二个单元格来获取你的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

#### 导入 TensorFlow

将 TensorFlow 包导入到您的 Python 环境中。

In [ ]:
import tensorflow as tf

#### 导入 TensorFlow Transform

将 TensorFlow Transform（TFT）包导入到您的 Python 环境中。

In [ ]:
import tensorflow_transform as tft

#### 导入 TensorFlow 数据验证

将 TensorFlow 数据验证（TFDV）包导入您的 Python 环境中。

In [ ]:
import tensorflow_data_validation as tfdv

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 来使用支持 GPU 的容器映像以及分配给虚拟机实例的 GPU 数量。例如，要使用一个支持 GPU 的容器映像，并为每个虚拟机分配 4 个 Nvidia Telsa K80 GPU，则可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定 `(None, None)` 来使用一个在 CPU 上运行的容器映像。

了解更多关于 [您所在地区硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF 2.3 之前的版本在 GPU 支持上无法加载本教程中的自定义模型。这是一个已知问题，在 TF 2.3 中已修复。这是由生成在服务函数中的静态图操作引起的。如果您在自己的自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
import os

if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

设置用于训练和预测的预先构建的Docker容器镜像。

有关最新列表，请查看[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请查看[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
- `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 内存。
     - `n1-highcpu`: 每个 vCPU 0.9GB 内存。
- `vCPUs`: \[2, 4, 8, 16, 32, 64, 96 \] 中的数字。

*注: 以下机器类型不支持训练:*

- `standard`: 2 vCPUs
- `highcpu`: 2, 4 和 8 vCPUs

*提示: 您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 从阶段1检索数据集

接下来，使用辅助函数`find_dataset()`检索在阶段1创建的数据集。该辅助函数会查找所有显示名称与指定前缀和导入格式（例如，bq）匹配的数据集。最后，它会按创建时间对匹配项进行排序并返回最新版本。

In [ ]:
def find_dataset(display_name_prefix, import_format):
    matches = []
    datasets = aip.TabularDataset.list()
    for dataset in datasets:
        if dataset.display_name.startswith(display_name_prefix):
            try:
                if (
                    "bq" == import_format
                    and dataset.to_dict()["metadata"]["inputConfig"]["bigquerySource"]
                ):
                    matches.append(dataset)
                if (
                    "csv" == import_format
                    and dataset.to_dict()["metadata"]["inputConfig"]["gcsSource"]
                ):
                    matches.append(dataset)
            except:
                pass

    create_time = None
    for match in matches:
        if create_time is None or match.create_time > create_time:
            create_time = match.create_time
            dataset = match

    return dataset


dataset = find_dataset("Chicago Taxi", "bq")

print(dataset)

加载数据集的用户元数据

加载数据集的用户元数据。

In [ ]:
import json

try:
    with tf.io.gfile.GFile(
        "gs://" + dataset.labels["user_metadata"] + "/metadata.jsonl", "r"
    ) as f:
        metadata = json.load(f)

    print(metadata)
except:
    print("no metadata")

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行该管道。

#### 创建训练管道

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `optimization_prediction_type`：为模型训练的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLTabularTrainingJob(
    display_name="chicago_" + TIMESTAMP,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(dag)

运行训练管道

接下来，您可以通过调用方法`run`来运行DAG以启动训练作业，需要传入以下参数：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留置数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要训练为标签的列的名称。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则可能在服务认为无法进一步提高模型目标测量之前完成训练，并未使用整个预算。

完成`run`方法后将返回`Model`资源。

训练管道的执行将需要最多180分钟。

In [ ]:
async_model = dag.run(
    dataset=dataset,
    model_display_name="chicago_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column="tip_bin",
    sync=False,
)

### 创建用于跟踪与训练相关的元数据的实验

设置跟踪每个实验的参数（配置）和指标（结果）：

- `aip.init()` - 创建一个实验实例
- `aip.start_run()` - 跟踪实验中的特定运行。

了解更多关于[Vertex AI ML Metadata简介](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction)。

In [ ]:
EXPERIMENT_NAME = "chicago-" + TIMESTAMP
aip.init(experiment=EXPERIMENT_NAME)
aip.start_run("run-1")

创建一个 Vertex AI TensorBoard 实例，以便在自定义模型训练中与 Vertex AI Training 一起使用 TensorBoard。

了解更多关于 [开始使用 Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview)。

In [ ]:
TENSORBOARD_DISPLAY_NAME = "chicago_" + TIMESTAMP
tensorboard = aip.Tensorboard.create(display_name=TENSORBOARD_DISPLAY_NAME)
tensorboard_resource_name = tensorboard.gca_resource.name
print("TensorBoard resource name:", tensorboard_resource_name)

### 为您的自定义模型创建输入层

接下来，根据每个特征的数据类型为您的自定义表格模型创建输入层。

In [ ]:
from tensorflow.keras.layers import Input


def create_model_inputs(
    numeric_features=None, categorical_features=None, embedding_features=None
):
    inputs = {}
    for feature_name in numeric_features:
        inputs[feature_name] = Input(name=feature_name, shape=[], dtype=tf.float32)
    for feature_name in categorical_features:
        inputs[feature_name] = Input(name=feature_name, shape=[], dtype=tf.int64)
    for feature_name in embedding_features:
        inputs[feature_name] = Input(name=feature_name, shape=[], dtype=tf.int64)

    return inputs

In [ ]:
input_layers = create_model_inputs(
    numeric_features=metadata["numeric_features"],
    categorical_features=metadata["categorical_features"],
    embedding_features=metadata["embedding_features"],
)

print(input_layers)

### 创建二分类器自定义模型

接下来，您将创建您的二分类器自定义表格模型。

In [ ]:
from math import sqrt

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import (Activation, Concatenate, Dense, Embedding,
                                     experimental)


def create_binary_classifier(
    input_layers,
    tft_output,
    metaparams,
    numeric_features,
    categorical_features,
    embedding_features,
):
    layers = []
    for feature_name in input_layers:
        if feature_name in embedding_features:
            vocab_size = tft_output.vocabulary_size_by_name(feature_name)
            embedding_size = int(sqrt(vocab_size))
            embedding_output = Embedding(
                input_dim=vocab_size + 1,
                output_dim=embedding_size,
                name=f"{feature_name}_embedding",
            )(input_layers[feature_name])
            layers.append(embedding_output)
        elif feature_name in categorical_features:
            vocab_size = tft_output.vocabulary_size_by_name(feature_name)
            onehot_layer = experimental.preprocessing.CategoryEncoding(
                num_tokens=vocab_size,
                output_mode="binary",
                name=f"{feature_name}_onehot",
            )(input_layers[feature_name])
            layers.append(onehot_layer)
        elif feature_name in numeric_features:
            numeric_layer = tf.expand_dims(input_layers[feature_name], -1)
            layers.append(numeric_layer)
        else:
            pass

    joined = Concatenate(name="combines_inputs")(layers)
    feedforward_output = Sequential(
        [Dense(units, activation="relu") for units in metaparams["hidden_units"]],
        name="feedforward_network",
    )(joined)
    logits = Dense(units=1, name="logits")(feedforward_output)
    pred = Activation("sigmoid")(logits)

    model = Model(inputs=input_layers, outputs=[pred])
    return model

In [ ]:
TRANSFORM_ARTIFACTS_DIR = metadata["transform_artifacts_dir"]
tft_output = tft.TFTransformOutput(TRANSFORM_ARTIFACTS_DIR)

metaparams = {"hidden_units": [128, 64]}
aip.log_params(metaparams)

model = create_binary_classifier(
    input_layers,
    tft_output,
    metaparams,
    numeric_features=metadata["numeric_features"],
    categorical_features=metadata["categorical_features"],
    embedding_features=metadata["embedding_features"],
)

model.summary()

接下来，可视化自定义模型的架构。

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True)

### 保存模型文件

接下来，将模型文件保存到您的云存储桶中。

In [ ]:
MODEL_DIR = f"{BUCKET_NAME}/base_model"

model.save(MODEL_DIR)

### 将本地模型上传到 Vertex AI 模型资源

接下来，您将上传本地自定义模型构件到 Vertex AI，以便转换为托管的 Vertex AI 模型资源。

In [ ]:
vertex_custom_model = aip.Model.upload(
    display_name="chicago_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    labels={"base_model": "1"},
    sync=True,
)

### 构建培训包

#### 包布局

在开始培训之前，您应该了解如何为自定义培训任务组装一个Python包。解压后，该包包含以下目录/文件布局:

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py
  - 其他Python脚本

文件`setup.cfg`和`setup.py`是将包安装到Docker映像的操作环境中的说明。

文件`trainer/task.py`是执行自定义培训任务的Python脚本。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'google-cloud-aiplatform',\n\n        'cloudml-hypertune',\n\n        'tensorflow_datasets==1.3.0',\n\n        'tensorflow==2.5',\n\n    'tensorflow_data_validation==1.2',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Chicago Taxi tabular binary classifier\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: cdpe@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

获取预处理数据的特征规范

接下来，为预处理数据创建特征规范。

In [ ]:
transform_feature_spec = tft_output.transformed_feature_spec()
print(transform_feature_spec)

### 将转换后的数据加载到 tf.data.Dataset 中

接下来，您将在 Cloud Storage 存储中加载 gzip TFRecords 到一个 `tf.data.Dataset` 生成器中。这些功能在使用 `Vertex Training` 训练自定义模型时会被重复使用，因此您可以将它们保存到 Python 训练包中。

In [ ]:
%%writefile custom/trainer/data.py

import tensorflow as tf

def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def get_dataset(file_pattern, feature_spec, label_column, batch_size=200):
    """Generates features and label for tuning/training.
    Args:
      file_pattern: input tfrecord file pattern.
      feature_spec: a dictionary of feature specifications.
      batch_size: representing the number of consecutive elements of returned
        dataset to combine in a single batch
    Returns:
      A dataset that contains (features, indices) tuple where features is a
        dictionary of Tensors, and indices is a single Tensor of label indices.
    """

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=feature_spec,
        label_key=label_column,
        reader=_gzip_reader_fn,
        num_epochs=1,
        drop_final_batch=True,
    )

    return dataset

In [ ]:
from custom.trainer import data

TRANSFORMED_DATA_PREFIX = metadata["transformed_data_prefix"]
LABEL_COLUMN = metadata["label_column"]

train_data_file_pattern = TRANSFORMED_DATA_PREFIX + "/train/data-*.gz"
val_data_file_pattern = TRANSFORMED_DATA_PREFIX + "/val/data-*.gz"
test_data_file_pattern = TRANSFORMED_DATA_PREFIX + "/test/data-*.gz"

for input_features, target in data.get_dataset(
    train_data_file_pattern, transform_feature_spec, LABEL_COLUMN, batch_size=3
).take(1):
    for key in input_features:
        print(
            f"{key} {input_features[key].dtype}: {input_features[key].numpy().tolist()}"
        )
    print(f"target: {target.numpy().tolist()}")

用转换后的输入测试模型架构。

接下来，使用转换后的训练输入样本测试模型架构。

*注意:*由于模型尚未训练，预测结果应该是随机的。由于这是一个二元分类器，预期预测结果约为0.5。

In [ ]:
model(input_features)

## 开发和测试训练脚本

在进行实验时，通常会先在本地开发和测试训练包，然后再转移到云端进行训练。

### 创建训练脚本

接下来，您需要编写Python脚本来编译和训练模型。

In [ ]:
%%writefile custom/trainer/train.py

from trainer import data
import tensorflow as tf
import logging
from hypertune import HyperTune

def compile(model, hyperparams):
    ''' Compile the model '''
    optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparams["learning_rate"])
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    metrics = [tf.keras.metrics.BinaryAccuracy(name="accuracy")]

    model.compile(optimizer=optimizer,loss=loss, metrics=metrics)
    return model

def warmup(
    model,
    hyperparams,
    train_data_dir,
    label_column,
    transformed_feature_spec
):
    ''' Warmup the initialized model weights '''

    train_dataset = data.get_dataset(
        train_data_dir,
        transformed_feature_spec,
        label_column,
        batch_size=hyperparams["batch_size"],
    )

    lr_inc = (hyperparams['end_learning_rate'] - hyperparams['start_learning_rate']) / hyperparams['num_epochs']

    def scheduler(epoch, lr):
        if epoch == 0:
            return hyperparams['start_learning_rate']
        return lr + lr_inc


    callbacks = [tf.keras.callbacks.LearningRateScheduler(scheduler)]

    logging.info("Model warmup started...")
    history = model.fit(
            train_dataset,
            epochs=hyperparams["num_epochs"],
            steps_per_epoch=hyperparams["steps"],
            callbacks=callbacks
    )

    logging.info("Model warmup completed.")
    return history


def train(
    model,
    hyperparams,
    train_data_dir,
    val_data_dir,
    label_column,
    transformed_feature_spec,
    log_dir,
    tuning=False
):
    ''' Train the model '''

    train_dataset = data.get_dataset(
        train_data_dir,
        transformed_feature_spec,
        label_column,
        batch_size=hyperparams["batch_size"],
    )

    val_dataset = data.get_dataset(
        val_data_dir,
        transformed_feature_spec,
        label_column,
        batch_size=hyperparams["batch_size"],
    )

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor=hyperparams["early_stop"]["monitor"], patience=hyperparams["early_stop"]["patience"], restore_best_weights=True
    )

    callbacks = [early_stop]

    if log_dir:
        tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

        callbacks = callbacks.append(tensorboard)

    if tuning:
        # Instantiate the HyperTune reporting object
        hpt = HyperTune()

        # Reporting callback
        class HPTCallback(tf.keras.callbacks.Callback):

            def on_epoch_end(self, epoch, logs=None):
                hpt.report_hyperparameter_tuning_metric(
                    hyperparameter_metric_tag='val_loss',
                    metric_value=logs['val_loss'],
                    global_step=epoch
                )

        if not callbacks:
            callbacks = []
        callbacks.append(HPTCallback())

    logging.info("Model training started...")
    history = model.fit(
            train_dataset,
            epochs=hyperparams["num_epochs"],
            validation_data=val_dataset,
            callbacks=callbacks
    )

    logging.info("Model training completed.")
    return history

def evaluate(
    model,
    hyperparams,
    test_data_dir,
    label_column,
    transformed_feature_spec
):
    logging.info("Model evaluation started...")
    test_dataset = data.get_dataset(
        test_data_dir,
        transformed_feature_spec,
        label_column,
        hyperparams["batch_size"],
    )

    evaluation_metrics = model.evaluate(test_dataset)
    logging.info("Model evaluation completed.")

    return evaluation_metrics

### 在本地训练模型

接下来，通过仅训练几个时期来本地测试训练包：

- `num_epochs`：传递给训练包的时期数。
- `compile()`: 编译模型以进行训练。
- `warmup()`: 预热初始化的模型权重。
- `train()`: 训练模型。

In [ ]:
os.chdir("custom")

import logging

from trainer import train

TENSORBOARD_LOG_DIR = "./logs"

logging.getLogger().setLevel(logging.INFO)

hyperparams = {}
hyperparams["learning_rate"] = 0.01
aip.log_params(hyperparams)

train.compile(model, hyperparams)

warmupparams = {}
warmupparams["start_learning_rate"] = 0.0001
warmupparams["end_learning_rate"] = 0.01
warmupparams["num_epochs"] = 4
warmupparams["batch_size"] = 64
warmupparams["steps"] = 50
aip.log_params(warmupparams)

train.warmup(
    model, warmupparams, train_data_file_pattern, LABEL_COLUMN, transform_feature_spec
)

trainparams = {}
trainparams["num_epochs"] = 5
trainparams["batch_size"] = 64
trainparams["early_stop"] = {"monitor": "val_loss", "patience": 5}
aip.log_params(trainparams)

train.train(
    model,
    trainparams,
    train_data_file_pattern,
    val_data_file_pattern,
    LABEL_COLUMN,
    transform_feature_spec,
    TENSORBOARD_LOG_DIR,
)

os.chdir("..")

### 在本地评估模型

接下来，测试训练包的评估部分：

- `evaluate()`: 评估模型。

In [ ]:
os.chdir("custom")

from trainer import train

evalparams = {}
evalparams["batch_size"] = 64

metrics = {}
metrics["loss"], metrics["acc"] = train.evaluate(
    model, evalparams, test_data_file_pattern, LABEL_COLUMN, transform_feature_spec
)
print("ACC", metrics["acc"], "LOSS", metrics["loss"])
aip.log_metrics(metrics)

os.chdir("..")

### 从 Vertex AI 检索模型

接下来，创建 Python 脚本以从 Vertex AI 检索您的实验模型。

In [ ]:
%%writefile custom/trainer/model.py

import google.cloud.aiplatform as aip

def get(model_id):
    model = aip.Model(model_id)
    return model

### 为 Python 培训包创建任务脚本

接下来，您需要为驱动培训包创建 `task.py` 脚本。一些值得注意的步骤包括：

- 命令行参数：
    - `model-id`：在实验期间构建的 `Model` 资源的资源 ID。这是未经训练的模型架构。
    - `dataset-id`：用于训练的 `Dataset` 资源的资源 ID。
    - `experiment`：实验的名称。
    - `run`：实验中的运行的名称。
    - `tensorboard-logdir`：Vertex AI Tensorboard 的日志目录。


- `get_data()`：
    - 将 Dataset 资源加载到内存中。
    - 从 Dataset 资源获取用户元数据。
    - 从元数据中获取转化数据的位置、转化功能以及标签列的名称。


- `get_model()`：
    - 将 Model 资源加载到内存中。
    - 获取模型架构的模型工件的位置。
    - 加载模型架构。
    - 编译模型。


- `warmup_model()`：
   - 对初始化的模型权重进行预热。


- `train_model()`：
    - 训练模型。


- `evaluate_model()`：
    - 评估模型。
    - 将评估指标保存到 Cloud Storage 存储桶中。

In [ ]:
%%writefile custom/trainer/task.py
import os
import argparse
import logging
import json

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.python.client import device_lib

import google.cloud.aiplatform as aip

from trainer import data
from trainer import model as model_
from trainer import train
try:
    from trainer import serving
except:
    pass

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--model-id', dest='model_id',
                    default=None, type=str, help='Vertex Model ID.')
parser.add_argument('--dataset-id', dest='dataset_id',
                    default=None, type=str, help='Vertex Dataset ID.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--start_lr', dest='start_lr',
                    default=0.0001, type=float,
                    help='Starting learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--batch_size', dest='batch_size',
                    default=16, type=int,
                    help='Batch size.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--tensorboard-log-dir', dest='tensorboard_log_dir',
                    default=os.getenv('AIP_TENSORBOARD_LOG_DIR'), type=str,
                    help='Output file for tensorboard logs')
parser.add_argument('--experiment', dest='experiment',
                    default=None, type=str,
                    help='Name of experiment')
parser.add_argument('--project', dest='project',
                    default=None, type=str,
                    help='Name of project')
parser.add_argument('--run', dest='run',
                    default=None, type=str,
                    help='Name of run in experiment')
parser.add_argument('--evaluate', dest='evaluate',
                    default=False, type=bool,
                    help='Whether to perform evaluation')
parser.add_argument('--serving', dest='serving',
                    default=False, type=bool,
                    help='Whether to attach the serving function')
parser.add_argument('--tuning', dest='tuning',
                    default=False, type=bool,
                    help='Whether to perform hyperparameter tuning')
parser.add_argument('--warmup', dest='warmup',
                    default=False, type=bool,
                    help='Whether to perform warmup weight initialization')
args = parser.parse_args()


logging.getLogger().setLevel(logging.INFO)
logging.info('DEVICES'  + str(device_lib.list_local_devices()))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
    logging.info("Single device training")
# Single Machine, multiple compute device
elif args.distribute == 'mirrored':
    strategy = tf.distribute.MirroredStrategy()
    logging.info("Mirrored Strategy distributed training")
# Multi Machine, multiple compute device
elif args.distribute == 'multiworker':
    strategy = tf.distribute.MultiWorkerMirroredStrategy()
    logging.info("Multi-worker Strategy distributed training")
    logging.info('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
logging.info('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Initialize the run for this experiment
if args.experiment:
    logging.info("Initialize experiment: {}".format(args.experiment))
    aip.init(experiment=args.experiment, project=args.project)
    aip.start_run(args.run)

metadata = {}

def get_data():
    ''' Get the preprocessed training data '''
    global train_data_file_pattern, val_data_file_pattern, test_data_file_pattern
    global label_column, transform_feature_spec, metadata

    dataset = aip.TabularDataset(args.dataset_id)
    METADATA = 'gs://' + dataset.labels['user_metadata'] + "/metadata.jsonl"

    with tf.io.gfile.GFile(METADATA, "r") as f:
        metadata = json.load(f)

    TRANSFORMED_DATA_PREFIX = metadata['transformed_data_prefix']
    label_column = metadata['label_column']

    train_data_file_pattern = TRANSFORMED_DATA_PREFIX + '/train/data-*.gz'
    val_data_file_pattern = TRANSFORMED_DATA_PREFIX + '/val/data-*.gz'
    test_data_file_pattern = TRANSFORMED_DATA_PREFIX + '/test/data-*.gz'

    TRANSFORM_ARTIFACTS_DIR = metadata['transform_artifacts_dir']
    tft_output = tft.TFTransformOutput(TRANSFORM_ARTIFACTS_DIR)
    transform_feature_spec = tft_output.transformed_feature_spec()

def get_model():
    ''' Get the untrained model architecture '''
    global model_artifacts

    vertex_model = model_.get(args.model_id)
    model_artifacts = vertex_model.gca_resource.artifact_uri
    model = tf.keras.models.load_model(model_artifacts)

    # Compile the model
    hyperparams = {}
    hyperparams["learning_rate"] = args.lr
    if args.experiment:
        aip.log_params(hyperparams)

    metadata.update(hyperparams)
    with tf.io.gfile.GFile(os.path.join(args.model_dir, "metrics.txt"), "w") as f:
        f.write(json.dumps(metadata))

    train.compile(model, hyperparams)
    return model

def warmup_model(model):
    ''' Warmup the initialized model weights '''
    warmupparams = {}
    warmupparams["num_epochs"] = args.epochs
    warmupparams["batch_size"] = args.batch_size
    warmupparams["steps"] = args.steps
    warmupparams["start_learning_rate"] = args.start_lr
    warmupparams["end_learning_rate"] = args.lr

    train.warmup(model, warmupparams, train_data_file_pattern, label_column, transform_feature_spec)
    return model

def train_model(model):
    ''' Train the model '''
    trainparams = {}
    trainparams["num_epochs"] = args.epochs
    trainparams["batch_size"] = args.batch_size
    trainparams["early_stop"] = {"monitor": "val_loss", "patience": 5}
    if args.experiment:
        aip.log_params(trainparams)

    metadata.update(trainparams)
    with tf.io.gfile.GFile(os.path.join(args.model_dir, "metrics.txt"), "w") as f:
        f.write(json.dumps(metadata))

    train.train(model, trainparams, train_data_file_pattern, val_data_file_pattern, label_column, transform_feature_spec, args.tensorboard_log_dir, args.tuning)
    return model

def evaluate_model(model):
    ''' Evaluate the model '''
    evalparams = {}
    evalparams["batch_size"] = args.batch_size
    metrics = train.evaluate(model, evalparams, test_data_file_pattern, label_column, transform_feature_spec)

    metadata.update({'metrics': metrics})
    with tf.io.gfile.GFile(os.path.join(args.model_dir, "metrics.txt"), "w") as f:
        f.write(json.dumps(metadata))

get_data()
with strategy.scope():
    model = get_model()

if args.warmup:
    model = warmup_model(model)
else:
    model = train_model(model)

if args.evaluate:
    evaluate_model(model)

if args.serving:
    logging.info('Save serving model to: ' + args.model_dir)
    serving.construct_serving_model(
        model=model,
        serving_model_dir=args.model_dir,
        metadata=metadata
    )
elif args.warmup:
    logging.info('Save warmed up model to: ' + model_artifacts)
    model.save(model_artifacts)
else:
    logging.info('Save trained model to: ' + args.model_dir)
    model.save(args.model_dir)

### 本地测试培训套餐

接下来，只需进行几个周期的本地测试，测试您完成的培训套餐。

In [ ]:
DATASET_ID = dataset.resource_name
MODEL_ID = vertex_custom_model.resource_name
!cd custom; python3 -m trainer.task --model-id={MODEL_ID} --dataset-id={DATASET_ID} --experiment='chicago' --run='test' --project={PROJECT_ID} --epochs=5 --model-dir=/tmp --evaluate=True

热身训练

现在你已经测试了训练脚本，你可以在基础模型上进行热身训练。热身训练是用来稳定权重初始化的。通过这样做，每次对模型架构进行训练和调整都将从相同稳定的权重初始化开始。

In [ ]:
MODEL_DIR = f"{BUCKET_NAME}/base_model"

!cd custom; python3 -m trainer.task --model-id={MODEL_ID} --dataset-id={DATASET_ID} --project={PROJECT_ID} --epochs=5 --steps=300 --batch_size=16 --lr=0.01 --start_lr=0.0001 --model-dir={MODEL_DIR} --warmup=True

## 镜像策略

在单个VM上进行训练时，可以选择在单个计算设备上训练，也可以选择在同一VM上的多个计算设备上训练。使用Vertex AI Distributed Training，您可以指定VM实例的计算设备数量和计算设备类型：CPU，GPU。

Vertex AI Distributed Training支持TensorFlow模型的`tf.distribute.MirroredStrategy'。要在同一VM上多个计算设备上进行训练，您需要在Python训练脚本中执行以下额外步骤：

1. 设置`tf.distribute.MirrorStrategy`
2. 在`tf.distribute.MirrorStrategy`的范围内编译模型。*注意：*告诉MirroredStrategy要在计算设备之间镜像哪些变量。
3. 将每个计算设备的批量大小增加到num_devices * batch size。

在过渡期间，每个批次的分布以及对模型参数的更新将被同步。

### 创建和运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练作业，2）运行该作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，需要以下参数：

- `display_name`：自定义训练作业的人类可读名称。
- `container_uri`：训练容器镜像。
- `python_package_gcs_uri`：Python训练包的位置，以tarball格式。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

*注意：* 没有requirements参数。您可以在Python包的`setup.py`脚本中指定任何需求。

In [ ]:
DISPLAY_NAME = "chicago_" + TIMESTAMP

job = aip.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_NAME}/trainer_chicago.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

In [ ]:
! rm -rf custom/logs
! rm -rf custom/trainer/__pycache__

您可以将培训脚本存储在您的云存储桶中。

接下来，您将培训文件夹打包成压缩的tar包，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_chicago.tar.gz

#### 运行自定义 Python 包训练作业

接下来，您可以通过调用 `run()` 方法来运行自定义作业，从而开始训练作业。参数与运行 CustomTrainingJob 时相同。

*注意:* 参数 `service_account` 被设置为使得初始化实验步骤 `aip.init(experiment="...")` 必须具有权限访问 Vertex AI Metadata 存储库。

In [ ]:
MODEL_DIR = BUCKET_NAME + "/testing"

CMDARGS = [
    "--epochs=5",
    "--batch_size=16",
    "--distribute=mirrored",
    "--experiment=chicago",
    "--run=test",
    "--project=" + PROJECT_ID,
    "--model-id=" + MODEL_ID,
    "--dataset-id=" + DATASET_ID,
]

model = job.run(
    model_display_name="chicago_" + TIMESTAMP,
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    accelerator_type=TRAIN_GPU.name,
    accelerator_count=TRAIN_NGPU,
    base_output_dir=MODEL_DIR,
    service_account=SERVICE_ACCOUNT,
    tensorboard=tensorboard_resource_name,
    sync=True,
)

删除自定义训练作业

在训练作业完成后，您可以使用方法 `delete()` 删除训练作业。在完成之前，可以使用方法 `cancel()` 取消训练作业。

In [ ]:
job.delete()

删除模型

`delete()` 方法将删除模型。

In [ ]:
model.delete()

超参数调整

接下来，您可以使用训练包进行超参数调整。训练包有一些新增内容，使得同一个包可以用于超参数调整、本地测试以及完整的云端训练：

- 命令行：
  - `tuning`：指示在训练过程中使用HyperTune服务作为回调。

- `train()`：如果设置了tuning，将创建并添加一个回调到HyperTune服务。

准备您的机器规格

现在为您的自定义训练工作定义机器规格。这告诉Vertex应为训练提供哪种类型的机器实例。
- `machine_type`：要预留的GCP实例类型 -- 例如，n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则您正在使用GPU；否则将使用CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这告诉 Vertex 在培训中为每台机器实例提供何种类型和大小的磁盘。

- `boot_disk_type`：SSD 或标准。SSD 更快，标准更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘大小（单位：GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 为超参数调整作业定义工作池规范

接下来，定义工作池规范。请注意，我们计划调整学习率和批量大小，所以您不需要将它们作为命令行参数传递（已省略）。Vertex AI超参数调整服务将在试验期间为学习率和批量大小选择数值，并将它们作为命令行参数传递。

In [ ]:
CMDARGS = [
    "--epochs=5",
    "--distribute=mirrored",
    # "--experiment=chicago",
    # "--run=tune",
    # "--project=" + PROJECT_ID,
    "--model-id=" + MODEL_ID,
    "--dataset-id=" + DATASET_ID,
    "--tuning=True",
]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_chicago.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

## 创建自定义作业

使用类`CustomJob`创建自定义作业，例如用于超参数调整，具有以下参数：

- `display_name`：自定义作业的人类可读名称。
- `worker_pool_specs`：相应VM实例的规范。

In [ ]:
job = aip.CustomJob(
    display_name="chicago_" + TIMESTAMP, worker_pool_specs=worker_pool_spec
)

创建超参数调优作业

使用类`HyperparameterTuningJob`来创建一个超参数调优作业，具有以下参数：

- `display_name`：自定义作业的可读名称。
- `custom_job`：此自定义作业的工作池规范适用于所有试验中创建的CustomJobs。
- `metrics_spec`：要优化的指标。字典键是 metric_id，该 metric_id 由您的训练作业报告，字典值是指标的优化目标（'最小化'或'最大化'）。
- `parameter_spec`：要优化的参数。字典键是 metric_id，作为命令行关键字参数传递到您的训练作业中，字典值是指标的参数规格。
- `search_algorithm`：要使用的搜索算法：`grid`，`random`和`None`。如果指定`None`，则使用`Vizier`服务（贝叶斯）。
- `max_trial_count`：要执行的最大试验次数。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aip.HyperparameterTuningJob(
    display_name="chicago_" + TIMESTAMP,
    custom_job=job,
    metric_spec={
        "val_loss": "minimize",
    },
    parameter_spec={
        "lr": hpt.DoubleParameterSpec(min=0.001, max=0.1, scale="log"),
        "batch_size": hpt.DiscreteParameterSpec([16, 32, 64, 128, 256], scale="linear"),
    },
    search_algorithm=None,
    max_trial_count=8,
    parallel_trial_count=1,
)

运行超参数调整作业

使用`run()`方法执行超参数调整作业。

In [ ]:
hpt_job.run()

现在看看哪个试验是最好的：### 最佳试验

In [ ]:
best = (None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

print(best)

### 删除超参数调整任务

方法'delete()'将删除超参数调整任务。

In [ ]:
hpt_job.delete()

保存最佳的超参数值。

In [ ]:
LR = best[2]
BATCH_SIZE = int(best[1])

### 创建和运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练作业，2）运行该作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，其中包含以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。
- `python_package_gcs_uri`：Python训练包的位置，以tarball方式存储。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

*注意：* 没有`requirements`参数。您可以在Python包的`setup.py`脚本中指定任何依赖项。

In [ ]:
DISPLAY_NAME = "chicago_" + TIMESTAMP

job = aip.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_NAME}/trainer_chicago.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

运行自定义的Python包训练作业

接下来，通过调用方法`run()`来运行自定义作业以启动训练作业。参数与运行CustomTrainingJob时相同。

*注意:* 参数service_account 设置为使初始化实验步骤`aip.init(experiment="...")`具有访问Vertex AI Metadata Store权限。

In [ ]:
MODEL_DIR = BUCKET_NAME + "/trained"
FULL_EPOCHS = 100

CMDARGS = [
    f"--epochs={FULL_EPOCHS}",
    f"--lr={LR}",
    f"--batch_size={BATCH_SIZE}",
    "--distribute=mirrored",
    "--experiment=chicago",
    "--run=full",
    "--project=" + PROJECT_ID,
    "--model-id=" + MODEL_ID,
    "--dataset-id=" + DATASET_ID,
    "--evaluate=True",
]

model = job.run(
    model_display_name="chicago_" + TIMESTAMP,
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    accelerator_type=TRAIN_GPU.name,
    accelerator_count=TRAIN_NGPU,
    base_output_dir=MODEL_DIR,
    service_account=SERVICE_ACCOUNT,
    tensorboard=tensorboard_resource_name,
    sync=True,
)

删除自定义训练作业

在训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，可以使用`cancel()`方法取消训练作业。

In [ ]:
job.delete()

获取实验结果

接下来，您可以将实验名称作为参数传递给方法 `get_experiment_df()`，以将实验结果获取为 pandas dataframe。

In [ ]:
EXPERIMENT_NAME = "chicago"

experiment_df = aip.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

## 审查定制模型评估结果

接下来，您可以审查集成到培训包中的评估度量标准。

In [ ]:
METRICS = MODEL_DIR + "/model/metrics.txt"
! gsutil cat $METRICS

### 删除TensorBoard实例

接下来，删除TensorBoard实例。

In [ ]:
tensorboard.delete()

In [ ]:
vertex_custom_model = model
model = tf.keras.models.load_model(MODEL_DIR + "/model")

## 添加一个serving功能

接下来，您可以为在线和批处理预测向您的模型添加一个serving功能。这样可以将预测请求以原始格式（未经处理）发送，可以是序列化的TF.Example或JSONL对象。然后，serving功能将预处理预测请求转换为模型所期望的转换格式。

In [ ]:
%%writefile custom/trainer/serving.py

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import logging

def _get_serve_features_fn(model, tft_output):
    """Returns a function that accept a dictionary of features and applies TFT."""

    model.tft_layer = tft_output.transform_features_layer()

    @tf.function
    def serve_features_fn(raw_features):
        """Returns the output to be used in the serving signature."""

        transformed_features = model.tft_layer(raw_features)
        probabilities = model(transformed_features)
        return {"scores": probabilities}


    return serve_features_fn

def _get_serve_tf_examples_fn(model, tft_output, feature_spec):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tft_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        for key in list(feature_spec.keys()):
            if key not in features:
                feature_spec.pop(key)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)
        probabilities = model(transformed_features)
        return {"scores": probabilities}

    return serve_tf_examples_fn

def construct_serving_model(
    model, serving_model_dir, metadata
):
    global features

    schema_location = metadata['schema']
    features = metadata['numeric_features'] + metadata['categorical_features'] + metadata['embedding_features']
    print("FEATURES", features)
    tft_output_dir = metadata["transform_artifacts_dir"]

    schema = tfdv.load_schema_text(schema_location)
    feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(schema).feature_spec

    tft_output = tft.TFTransformOutput(tft_output_dir)

    # Drop features that were not used in training
    features_input_signature = {
        feature_name: tf.TensorSpec(
            shape=(None, 1), dtype=spec.dtype, name=feature_name
        )
        for feature_name, spec in feature_spec.items()
        if feature_name in features
    }

    signatures = {
        "serving_default": _get_serve_features_fn(
            model, tft_output
        ).get_concrete_function(features_input_signature),
        "serving_tf_example": _get_serve_tf_examples_fn(
            model, tft_output, feature_spec
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ),
    }

    logging.info("Model saving started...")
    model.save(serving_model_dir, signatures=signatures)
    logging.info("Model saving completed.")

### 构建服务模型

现在构建服务模型并将服务模型存储到您的云存储桶中。

In [ ]:
os.chdir("custom")

from trainer import serving

SERVING_MODEL_DIR = BUCKET_NAME + "/serving_model"

serving.construct_serving_model(
    model=model, serving_model_dir=SERVING_MODEL_DIR, metadata=metadata
)

serving_model = tf.keras.models.load_model(SERVING_MODEL_DIR)

os.chdir("..")

### 使用tf.Example数据在本地测试服务模型

接下来，为tf.Example数据测试服务模型中的层接口。

In [ ]:
EXPORTED_TFREC_PREFIX = metadata["exported_tfrec_prefix"]
file_names = tf.data.TFRecordDataset.list_files(
    EXPORTED_TFREC_PREFIX + "/data-*.tfrecord"
)
for batch in tf.data.TFRecordDataset(file_names).batch(3).take(1):
    predictions = serving_model.signatures["serving_tf_example"](batch)
    for key in predictions:
        print(f"{key}: {predictions[key]}")

### 使用JSONL数据在本地测试服务模型

接下来，测试服务模型中用于JSONL数据的层接口。

In [ ]:
schema = tfdv.load_schema_text(metadata["schema"])
feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(schema).feature_spec

instance = {
    "dropoff_grid": "POINT(-87.6 41.9)",
    "euclidean": 2064.2696,
    "loc_cross": "",
    "payment_type": "Credit Card",
    "pickup_grid": "POINT(-87.6 41.9)",
    "trip_miles": 1.37,
    "trip_day": 12,
    "trip_hour": 6,
    "trip_month": 2,
    "trip_day_of_week": 4,
    "trip_seconds": 555,
}

for feature_name in instance:
    dtype = feature_spec[feature_name].dtype
    instance[feature_name] = tf.constant([[instance[feature_name]]], dtype)

predictions = serving_model.signatures["serving_default"](**instance)
for key in predictions:
    print(f"{key}: {predictions[key].numpy()}")

### 将服务模型上传到 Vertex AI 模型资源

接下来，您将上传您的服务定制模型工件到 Vertex AI，以转换为托管的 Vertex AI 模型资源。

In [ ]:
vertex_serving_model = aip.Model.upload(
    display_name="chicago_" + TIMESTAMP,
    artifact_uri=SERVING_MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    labels={"user_metadata": BUCKET_NAME[5:]},
    sync=True,
)

### 评估服务模型

接下来，使用评估（测试）切片来评估服务模型。为了进行苹果对苹果的比较，您可以为自定义模型和AutoML模型使用相同的评估切片。由于您的评估切片和指标可能是自定义的，我们建议：

- 将每个评估切片作为 Vertex AI 批量预测作业发送。
- 使用自定义评估脚本来评估批量预测作业的结果。

In [ ]:
SERVING_OUTPUT_DATA_DIR = BUCKET_NAME + "/batch_eval"
EXPORTED_JSONL_PREFIX = metadata["exported_jsonl_prefix"]

MIN_NODES = 1
MAX_NODES = 1

job = vertex_serving_model.batch_predict(
    instances_format="jsonl",
    predictions_format="jsonl",
    job_display_name="chicago_" + TIMESTAMP,
    gcs_source=EXPORTED_JSONL_PREFIX + "*.jsonl",
    gcs_destination_prefix=SERVING_OUTPUT_DATA_DIR,
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=True,
)

### 执行自定义评估指标

在批处理作业完成后，您将结果和目标标签输入到您的自定义评估脚本中。为了演示目的，我们只显示批处理预测的结果。

In [ ]:
batch_dir = ! gsutil ls $SERVING_OUTPUT_DATA_DIR
batch_dir = batch_dir[0]
outputs = ! gsutil ls $batch_dir
errors = outputs[0]
results = outputs[1]
print("errors")
! gsutil cat $errors
print("results")
! gsutil cat $results | head -n10

In [ ]:
model = async_model

### 等待AutoML训练作业完成

接下来，等待AutoML训练作业完成。或者，可以在`run()`方法中将参数`sync`设置为`True`，以阻塞直到AutoML训练作业完成。

In [ ]:
model.wait()

## 检查模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations（）`方法来查看其评估分数。该方法将返回每个评估分片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

最后，根据以下步骤，您可以决定当前实验是否产生了比AutoML基准更好的自定义模型：
- 比较自定义模型和AutoML模型在每个评估切片上的评估结果。
- 根据您的业务目的对结果进行加权。
- 综合考虑结果并确定自定义模型是否更好。

### 存储自定义模型的店铺评估结果

接下来，您可以使用标签字段来存储包含自定义指标信息的用户元数据。

In [ ]:
import json

metadata = {}
metadata["train_eval_metrics"] = METRICS
metadata["custom_eval_metrics"] = "[you-fill-this-in]"

with tf.io.gfile.GFile("gs://" + BUCKET_NAME[5:] + "/metadata.jsonl", "w") as f:
    json.dump(metadata, f)

!gsutil cat $BUCKET_NAME/metadata.jsonl

清理

要清理此项目中使用的所有 Google 云资源，您可以删除用于本教程的 [Google 云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调优作业
- 云存储存储桶

In [ ]:
delete_all = False

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME